In [ ]:
import pandas as pd
import numpy as np
import os
import os.path
from os.path import join

In [ ]:
df_ref = pd.read_csv(r'T:\ABM\ABM_FY22\RSM\VisionEval\Model\Update_Automations\SrcOnSanVE\geo\geo.csv')
bzone_ref = df_ref[['Bzone','Marea']]

In [ ]:
def bzone_urban_town_du_proportions (bzone_ref):
    bzone_ref.columns = ['taz', 'Marea']
    input_path = r"T:\RTP\2021RP\2021rp_draft_v2\abm_runs" 
    years = ['2016', '2035build']
    data = []
    for year in years:
        mgra = pd.read_csv(os.path.join(input_path, year,'input', 'mgra13_based_input'+year[:4]+'.csv'),
                       usecols = ["taz", #TAZ number
                                 "mgra", #MGRA number
                                 "hs_sf", #number of single family units
                                 "hs_mf", #number of multi family units
                                 "gq_civ", #GQ civilian
                                 "gq_mil"]) #GQ military
        
        loc_type = pd.read_csv('2.3/areatype_mgra.csv')                                
        df = mgra.merge(loc_type[['MGRA','Loc_Type']], right_on = 'MGRA', left_on ='mgra', how = "left")
        df['UrbanSF'] = 0
        df.loc[df['Loc_Type'] == 'Urban', 'UrbanSF'] = df['hs_sf']
        df['UrbanMF'] = 0
        df.loc[df['Loc_Type'] == 'Urban', 'UrbanMF'] = df['hs_mf']
        df['UrbanGQ'] = 0
        df.loc[df['Loc_Type'] == 'Urban', 'UrbanGQ'] = df['gq_civ'] + df['gq_mil']
        df['TownSF'] = 0
        df.loc[df['Loc_Type'] == 'Town', 'TownSF'] = df['hs_sf']
        df['TownMF'] = 0
        df.loc[df['Loc_Type'] == 'Town', 'TownMF'] = df['hs_mf']
        df['TownGQ'] = 0
        df.loc[df['Loc_Type'] == 'Town', 'TownGQ'] = df['gq_civ'] + df['gq_mil']
        
        df2 = pd.DataFrame(df.groupby('taz')['UrbanSF','UrbanMF','UrbanGQ','TownSF','TownMF','TownGQ','hs_sf','hs_mf','gq_civ','gq_mil'].sum()).reset_index()
        df2 = bzone_ref[['taz']].merge(df2, on = 'taz', how = "left")
        df2['PropUrbanSFDU'] = df2['UrbanSF']/df2['hs_sf']
        df2['PropUrbanMFDU'] = df2['UrbanMF']/df2['hs_mf']
        df2['PropUrbanGQDU'] = df2['UrbanGQ']/(df2['gq_civ'] + df2['gq_mil'])
        df2['PropTownSFDU'] = df2['TownSF']/df2['hs_sf']
        df2['PropTownMFDU'] = df2['TownMF']/df2['hs_mf']
        df2['PropTownGQDU'] = df2['TownGQ']/(df2['gq_civ'] + df2['gq_mil'])
        df2['Year'] = year[:4]
        data.append(df2)
    df_final = pd.concat(data)
    df_final = df_final.rename({'taz': 'Geo'}, axis=1)
    df_final = df_final[['Geo','Year','PropUrbanSFDU','PropUrbanMFDU','PropUrbanGQDU','PropTownSFDU','PropTownMFDU','PropTownGQDU']].fillna(0)
    df_final.to_csv('bzone_urban-town_du_proportions.csv', index=False)

In [ ]:
import pyodbc
import pandas as pd
import geopandas as gpd
from shapely import wkt

def get_gis_lyr(sql, server, database):
    # establish SQL connection
    conn = pyodbc.connect("Driver={SQL Server};"
                          f"Server={server};"
                          f"Database={database};"
                          "Trusted_Connection=yes;")
    # run query
    resultDF = pd.read_sql_query(sql, conn)
    
    # convert to GeoDF
    resultDF["geometry"] = [wkt.loads(x) for x in resultDF["geometry"]]
    resultDF = gpd.GeoDataFrame(resultDF, crs="EPSG:2230", geometry="geometry")
    
    return resultDF

In [ ]:
def bzone_network_design ():
    input_path = r"T:\RTP\2021RP\2021rp_draft_v2\abm_runs" 
    years = ['2016', '2035build']
    data = []
    for year in years:
        mgra1 = pd.read_csv(os.path.join(input_path, year,'input', 'mgra13_based_input'+year[:4]+'.csv'),
                       usecols = ["taz", #TAZ number
                                  "mgra", #mgra number
                                 "totint"]) #total intersections
        # load MGRA from GeoDepot
        server = 'sql2014b8'
        database = 'GeoDepot'
        sql = ("SELECT [mgra], [Shape].ToString() AS [geometry]"
            f" FROM [GeoDepot].[gis].[mgra13]")
        mgra2 = get_gis_lyr(sql, server, database)
        mgra2['squaremile'] = mgra2.area * 0.00000003587 # convert square ft to square mile
        
        mgra = mgra1.merge(mgra2, on='mgra', how="left")
        nd = pd.DataFrame(mgra.groupby('taz')['squaremile','totint'].sum()).reset_index()
        nd['D3bpo4'] = nd['totint'] / nd['squaremile']
        nd['Year'] = year[:4]
        data.append(nd)
    df_final = pd.concat(data)
    df_final = df_final.rename({'taz': 'Geo'}, axis=1)
    df_final = df_final[['Geo','Year','D3bpo4']]
    df_final.to_csv('bzone_network_design.csv', index=False)

In [ ]:
def bzone_urban_mixed_use_prop (bzone_ref):
    bzone_ref.columns = ['taz', 'Marea']
    input_path = r"T:\RTP\2021RP\2021rp_draft_v2\abm_runs" 
    years = ['2016', '2035build']
    data = []
    for year in years:
        hh = pd.read_csv(os.path.join(input_path, year,'input', 'households.csv'),
                       usecols = ["hhid", #Unique Housheold ID
                                  "mgra", #mgra of household
                                 "taz"]) #taz of household
        loc_type = pd.read_csv('2.5/areatype_mgra.csv')                                
        df = hh.merge(loc_type[['MGRA','Loc_Type']], right_on = 'MGRA', left_on ='mgra', how = "left")
        df['IsUrbanMixNbrhd'] = 0
        df.loc[((df['Loc_Type'] == 'Urban') | (df['Loc_Type'] == 'Town')), 'IsUrbanMixNbrhd'] = 1 
        df1 = pd.DataFrame(df.groupby('taz')['hhid'].count()).reset_index()
        df2 = pd.DataFrame(df[df['IsUrbanMixNbrhd'] == 1].groupby('taz')['hhid'].count()).reset_index()
        #get bzone information
        df3 = bzone_ref[['taz']].drop_duplicates()
        df3 = df3.merge(df1, on = "taz", how = "left")
        df3 = df3.merge(df2, on = "taz", how = "left")
        df3 = df3.fillna(0)
        df3['MixUseProp'] = df3['hhid_y']/df3['hhid_x'] 
        df3['Year'] = year[:4]
        data.append(df3)
    df_final = pd.concat(data)
    df_final = df_final.fillna(0)
    df_final = df_final.rename({'taz': 'Geo'}, axis=1)
    df_final = df_final[['Geo','Year','MixUseProp']].sort_values(by=['Year','Geo'])
    df_final.to_csv('bzone_urban-mixed-use_prop.csv', index=False)

In [ ]:
def assignCarSvcAvailability_Bzone ():
    input_path = r"T:\RTP\2021RP\2021rp_draft_v2\abm_runs" 
    years = ['2016', '2035build']
    data = []
    for year in years:
        mgra = pd.read_csv(os.path.join(input_path, year,'input', 'mgra13_based_input'+year[:4]+'.csv'),
                       usecols = ["taz", #TAZ number
                                 "emp_total", #Total employment
                                  "effective_acres", #Effective acres in the mgra
                                 "pop"]) #total population
        
        mgra_taz = pd.DataFrame(mgra.groupby('taz')['pop','emp_total','effective_acres'].sum()).reset_index()
        mgra_taz['density'] = (mgra_taz['pop'] + mgra_taz['emp_total'])/(mgra_taz['effective_acres']*0.0015625)                      
        
        conditions = [((mgra_taz['density']>=0) & (mgra_taz['density']<5000)),
                      ((mgra_taz['density']>=5000) & (mgra_taz['density']<100000000))]
        choices = ['High','Low']
        mgra_taz['CarSvcLevel'] = np.select(conditions, choices, default='Low')
        mgra_taz['Year'] = year[:4]
        data.append(mgra_taz)
    df_final = pd.concat(data)
    df_final = df_final.rename({'taz': 'Geo'}, axis=1)
    df_final = df_final[['Geo','CarSvcLevel','Year']]
    df_final.to_csv('bzone_carsvc_availability.csv', index=False)